In [11]:
import yfinance as yf
import pandas as pd
from datetime import date, datetime,timedelta
ticker = 'AAPL'
today_date = date.today()
#ticker = input("Enter the ticker of which you would like to view the options: ")
ticker = yf.Ticker(ticker)
current_price = ticker.history(period = '1d')['Close'][0]
options = ticker.options
#Changing the datatype for options so we can substract the option's date to lastTradeDate to get the maturity time.
if options:
  option = ticker.option_chain(options[-1])
datetimee = datetime.strptime(options[-1],"%Y-%m-%d")
expdate = datetimee.date()
maturity = expdate - today_date
total_days = maturity.days
today_years = total_days/365
#saving the option's data in a dataframe.
df = pd.DataFrame(data = option[0])

df.head()



<ipython-input-11-b95302616424>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = ticker.history(period = '1d')['Close'][0]


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AAPL270115C00125000,2024-09-27 16:32:49+00:00,125.0,113.73,112.25,115.40,-1.769997,-1.532465,1,10,0.464941,True,REGULAR,USD
1,AAPL270115C00150000,2024-09-27 19:30:21+00:00,150.0,93.30,91.25,95.10,0.230003,0.247129,20,742,0.416052,True,REGULAR,USD
2,AAPL270115C00165000,2024-09-26 13:30:05+00:00,165.0,81.00,81.05,83.30,0.000000,0.000000,1,29,0.388251,True,REGULAR,USD
3,AAPL270115C00170000,2024-09-27 19:23:18+00:00,170.0,78.30,77.00,79.75,0.800003,1.032262,3,28,0.382285,True,REGULAR,USD
4,AAPL270115C00175000,2024-09-20 15:03:20+00:00,175.0,77.37,73.00,76.30,0.000000,0.000000,1,3,0.376776,True,REGULAR,USD


In [12]:
import numpy as np
from scipy.stats import norm
def BlackScholes(St,K,r,t,v,option_type = "call"):
  option_type = option_type.lower()
  St = float(St)
  K = float(K)
  r = float(r)
  t = float(t)
  v = float(v)

  d1 = (np.log(St/K))+(t*(r + 0.5*v**2))/ (v*np.sqrt(t))
  d2 = d1 - v*np.sqrt(t)
  if option_type == "call":
    option_price = norm.cdf(d1)*St - norm.cdf(d2)*K*np.exp(-r*t)
  elif option_type == "put":
    option_price = K*np.exp(-r*t)*norm.cdf(-d2) - St*norm.cdf(-d1)
  return option_price
'''St = 100  # Current stock price
K = 100  # Strike price
t = 1    # Time to maturity (1 year)
r = 0.05 # Risk-free interest rate (5%)
v = 0.2
print("call Value =", BlackScholes(St,K,r,t,v,"call"))
print("Put value = ", BlackScholes(St,K,r,t,v,"put"))'''

'St = 100  # Current stock price\nK = 100  # Strike price\nt = 1    # Time to maturity (1 year)\nr = 0.05 # Risk-free interest rate (5%)\nv = 0.2\nprint("call Value =", BlackScholes(St,K,r,t,v,"call"))\nprint("Put value = ", BlackScholes(St,K,r,t,v,"put"))'

In [18]:
import yfinance as yf
import pandas as pd

#df['PredictedPrice'] = BlackScholes(St = df.ask,K = df.strike, t= maturity , r = 3.74, v = df.impliedVolatility)
df['PredictedPrice'] = df.apply(lambda row: BlackScholes(St=current_price, K=row['strike'], r=0.0374, t=today_years, v=row['impliedVolatility'], option_type="call"), axis=1)
df.tail()



,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,PredictedPrice
18,AAPL270115C00300000,2024-09-27 18:24:35+00:00,300.0,15.66,14.25,16.20,-0.34,-2.125001,30,88,0.272773,False,REGULAR,USD,18.616327
19,AAPL270115C00310000,2024-09-27 19:27:17+00:00,310.0,13.45,12.35,14.10,0.05,0.373136,3,255,0.270332,False,REGULAR,USD,15.228208
20,AAPL270115C00320000,2024-09-27 19:57:39+00:00,320.0,11.51,10.30,12.15,0.06,0.524021,70,52,0.267235,False,REGULAR,USD,11.872636
21,AAPL270115C00330000,2024-09-23 17:03:57+00:00,330.0,9.97,8.65,10.55,0.00,0.000000,10,5,0.265388,False,REGULAR,USD,8.863673
22,AAPL270115C00340000,2024-09-27 18:24:35+00:00,340.0,8.74,7.55,9.95,0.74,9.249997,30,207,0.271614,False,REGULAR,USD,7.405303
